In [1]:
import pandas as pd
from hashlib import sha256
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, BinaryType

from core.data_objects.bronze.bronze_event_data_object import BronzeEventDataObject

In [2]:
spark = SparkSession.builder.appName("test_mno_event_data").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/14 13:18:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/14 13:18:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# create data with errors to check filters
data = {
    'user_id':['00', '1', '1','1','2', '2', '2', '3', '3', '3'],
    'timestamp': ['2023-01-01T00:12:00', None, '2023-01-01T00:00:00', '2023-01-01T00:03:00', 
                  '2023-05-01T00:00:00', '2023-05-01T01:00:00', '2023-05-01T01:00:00', 
                  '2023-01-01T00:03:00', 'abc', '2023-01-01T00:03:00'],
    'mcc': [254, 254, 254, 254, 254, 254, 254, 254, 300000, 254], 
    'cell_id': ['214030412038931', '214030412038931', '214030412038931', 
                '214030412038931', '214030412038935', None, None, None, '214030412038939', None], 
    'latitude': [-3.62958, -3.62954, -3.62958, -3.62954, None, -3.62959, -3.62950, None, None, 99999],
    'longitude': [40.51873, 40.51870, 40.51873, 40.51870, None, None, 40.51874, None, None, 9999],
    'loc_error': [100, 100, 100, 100, None, None, 100, None, None, None]
}

In [4]:
# some preprocessing
mno_data = pd.DataFrame(data)
mno_data['user_id'] = mno_data['user_id'].apply(lambda x: sha256(x.encode('utf-8')).digest())
mno_data.loc[0, 'user_id'] = None
mno_data = mno_data.replace({float('nan'): None})

In [5]:
PATH = '/opt/data/lakehouse/bronze/mno_events'
number_of_folders = 5 # logically not more than 30/31
spark_df = spark.createDataFrame(mno_data, schema = BronzeEventDataObject.SCHEMA)
for i in range(number_of_folders):
    spark_df.write.parquet(f"{PATH}/2023010{str(i+1)}", mode='overwrite')